[모델 매개변수 최적화하기](https://tutorials.pytorch.kr/beginner/basics/optimization_tutorial.html)

## 모델 매개변수 최적화하기

In [9]:
"""
데이터에 매개변수를 최적화하여 모델 학습하고, 검증하고, 테스트 진행
각 반복 단계:
    모델이 출력을 추출
    모델이 추측과 정답 사이의 오류를 계산
    매개변수에 대한 오류의 도함수 수집
    경사하강법을 사용해 파라미터들 최적화
"""

# 기본 코드
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

## 하이퍼파라미터

In [10]:
"""
모델 최적화 과정을 제어할 수 있는 조절 가능한 매개변수
서로 다른 하이퍼파라미터 값은 모델 학습과 수렴율에 영향 미침

학습 시에 정의하는 하이퍼파라미터
    1. 에폭 수: 데이터셋 반복 정도
    2. 배치 크기: 매개변수 갱신 전 신경망을 통해 전파된 데이터 샘플 수
    3. 학습률: 각 배치/에폭에서 매개변수 조절 비율
"""

learning_rate = 1e-3
batch_size = 64
epochs = 5

## 최적화 단계

epoch: 최적화 단계의 각 반복

* 학습 단계: 학습용 데이터셋 반복, 최적 매개변수로 수렴
* 검증/테스트 단계: 모델 성능 개선 확인

### 손실 함수

획득한 결과와 실제 값 사이의 틀린 정도를 측정하여 학습 중 이 값을 최소화

주어진 데이터 샘플을 입력으로 계산한 예측과 정답을 비교하여 손실을 계산

일반적인 손실함수

1. 회귀 문제의 `nn.MSELoss`
2. 분류 문제의 `nn.NLLLoss`
3. `nn.CrossEntropyLoss`: `nn.LogSoftmax` + `nn.NLLLoss`

In [11]:
# 손실 함수 초기화
loss_fn = nn.CrossEntropyLoss()

### 옵티마이저

최적화 알고리즘: 모델의 오류를 줄이기 위해 모델 매개변수를 조절하는 과정이 수행되는 방식을 정의

모든 최적화 절차는 `optimizer` 객체에 캡슐화된다

In [12]:
# 학습하려는 모델의 매개변수와 학습률 하이퍼파라미터를 등록하여 옵티마이저 초기화
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

학습 단계의 최적화

1. `optimizer.zero_grad`: 모델 매개변수의 변화도 재설정, 중복 계산을 막기 위해 명시적으로 0으로 설정
2. `loss.backward()`: 예측 손실을 역전파
3. `optimizer.step()`: 역전파 단계에서 수집된 변화도로 매개변수 조정

In [15]:
# 전체 구현

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)

    for batch, (X, y) in enumerate(dataloader):
        # 예측 및 손실 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred=model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}\n")


In [16]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}", "-"*20)
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1 --------------------
loss: 2.181453 [   64/60000]
loss: 2.176692 [ 6464/60000]
loss: 2.124108 [12864/60000]
loss: 2.145625 [19264/60000]
loss: 2.109811 [25664/60000]
loss: 2.039901 [32064/60000]
loss: 2.092655 [38464/60000]
loss: 2.004670 [44864/60000]
loss: 2.006806 [51264/60000]
loss: 1.959471 [57664/60000]
Test Error: 
 Accuracy: 56.1%, Avg loss: 1.944161

Epoch 2 --------------------
loss: 1.965254 [   64/60000]
loss: 1.944948 [ 6464/60000]
loss: 1.835641 [12864/60000]
loss: 1.884332 [19264/60000]
loss: 1.778961 [25664/60000]
loss: 1.713086 [32064/60000]
loss: 1.768803 [38464/60000]
loss: 1.643926 [44864/60000]
loss: 1.670038 [51264/60000]
loss: 1.576959 [57664/60000]
Test Error: 
 Accuracy: 61.4%, Avg loss: 1.577529

Epoch 3 --------------------
loss: 1.631402 [   64/60000]
loss: 1.597851 [ 6464/60000]
loss: 1.448930 [12864/60000]
loss: 1.526762 [19264/60000]
loss: 1.398388 [25664/60000]
loss: 1.383843 [32064/60000]
loss: 1.423208 [38464/60000]
loss: 1.321696 [44864/60000]